In [1]:
import numpy as np
import random
#np.random.seed(123)

In [2]:
def areBinaryWeights(weights):
    flag = True
    th = 1e-12
    for i in range(np.shape(weights),-1,-1):
        if(weights[i] > th and (1-weights[i])>th):
            flag = false
            break
            
    return flag

In [3]:
import uuid
import datetime

def empty_problem():
    problem = {}
    problem['type'] = 'None'
    problem['N'] = 0
    problem['outliers'] = []
    problem['priors'] = []
    problem['dof'] = 0
    #problem['time'] = now
    problem['time'] = datetime.datetime.now()
    # Unique identifier
    problem['uuid'] = uuid.uuid4().hex
    return problem
    

In [4]:
def isProblem(problem):
    mandatory_fields = {'type', 'N', 'outliers', 'priors', 'dof'}
    #flag = all(isfield(problem, mandatory_fields))
    flag = all(field in problem for field in mandatory_fields)
    #flag = all(field in problem for field in mandatory_fields)
    return flag

In [5]:
def rand_in(n,m, interval):
    a = interval[0]
    b = interval[1]
    r = a + (b-a)*np.random. rand(n,m)
    
    return r

In [6]:
def randn_in(n,m, interval):
    a = interval[0]
    b = interval[1]
    r = a + (b-a)*np.random.randn(n,m)
    return r

In [7]:
from numpy.linalg import matrix_rank
import random

def random_matrix(n, m, desired_rank):
    max_rank = min(n, m)
    if desired_rank==None:
        desired_rank = max_rank
    assert desired_rank<=max_rank, 'Cannot generate a matrix a random matrix with the desired rank'
    M = 3*np.random.rand(n, m)
    while matrix_rank(M) < max_rank and matrix_rank(M) > 0:
        M = rand_in(n, m, [-1, 1])
    rank_M = matrix_rank(M)
    
    if rank_M != desired_rank:
        col_perm = random.sample(range(m), rank_M - desired_rank + 1)
        for i in range(rank_M-desired_rank+1):
                M[:, col_perm[0]] = M[:, col_perm[i]]
    return M

In [8]:
def gncWeightsUpdate(weights, mu, residuals, barc2):
    th1 = (mu+1)/mu*barc2
    th2 = (mu)/(mu+1)*barc2
    #print('mu is: ', mu)
    #print('th1 is: ', th1)
    #print('th2 is: ', th2)
    
    for k in range(np.shape(residuals)):
        if residuals[k] - th1 >=0:
            weights[k] = 0
        elif residuals[k] - th2 <=0:
            weights[k] = 1
        else:
            weights[k] = np.sqrt(barc2*mu*(mu+1)/residuals[k]) - mu
            #assert weights[k] >=0 and weights[k] <=1, 'weights calculation wrong!', weights(k)
            assert weights[k] >=0 and weights[k] <=1, 'weights calculation wrong!'
  
    return weights, th1, th2     
    

In [9]:
from scipy import sparse
from scipy.sparse.linalg import spsolve
import numpy as np
import numbers
import random
from scipy.stats import chi2

In [10]:
def linearRegressionProblem(N, outliers_percentage, **kwargs):
    
    StateDimension = kwargs.get('StateDimension', 3)
    #print(StateDimension, type(StateDimension))
    #print(isinstance(StateDimension, numbers.Number))
    #print(np.size(StateDimension), np.size(StateDimension)==1) 
    #print(StateDimension >=1) 
    
    assert isinstance(StateDimension, numbers.Number) and np.size(StateDimension)==1 and StateDimension >=1, 'StateDimension must be numeric, scalar, and >=1'
    
    MeasurementDimension = kwargs.get('MeasurementDimension', 3)
    assert isinstance(MeasurementDimension, numbers.Number) and np.size(MeasurementDimension)==1 and np.array(MeasurementDimension)>=1, 'MeasurementDimension must be numeric, scalar, and >=1'
    
    StateMeanandStd = kwargs.get('StateMeanAndStd', [1,1])
    assert all(isinstance(elem, numbers.Number) for elem in StateMeanandStd) and np.size(StateMeanandStd)==2 and all(np.array(StateMeanandStd) >=0), 'StateMeanandStd should be numeric, vector of length 2, and >=0'
    
    MeasurementNoiseStd = kwargs.get('MeasurementNoiseStd', 0.1)
    assert isinstance(MeasurementNoiseStd, numbers.Number) and np.size(MeasurementNoiseStd)==1 and MeasurementNoiseStd>0
    
    OutlierSpace = kwargs.get('OutlierSpace', [-100, 100])
    assert all(isinstance(elem, numbers.Number) for elem in OutlierSpace) and np.size(OutlierSpace)==2 and np.array(OutlierSpace[0]) <= np.array(OutlierSpace[1]), 'OutlierSpace must be numeric, vector of length 2, and OutlierSpace[0] < OutlierSpace[1]'
    
    if kwargs==None:
        outliers_percentage = 0
        
    problem = empty_problem()
    problem['N'] = N
    problem['StateDimension'] = StateDimension
    problem['MeasurementDimension'] = MeasurementDimension
    problem['MeasurementNoiseStd'] = MeasurementNoiseStd
    problem['dof'] = MeasurementDimension
    problem['type'] = 'linear-estimation'
    num_outliers = np.floor(outliers_percentage*N)
    problem['x_gt'] = np.random.normal(StateMeanandStd[0], StateMeanandStd[1], (StateDimension,1))
    
    problem['A'] = []
    for i in range(N):
        problem['A'].append(random_matrix(MeasurementDimension, StateDimension, min(MeasurementDimension, StateDimension)))
    #print('MeasurementDimension: ', MeasurementDimension)
    problem['y'] = []
    for i in range(N):
        problem['y'].append(problem['A'][i] .dot( problem['x_gt']) +  np.random.normal(0, int(MeasurementNoiseStd), size=(int(MeasurementDimension), 1)))
    
    convi = 1/MeasurementNoiseStd*np.eye(MeasurementDimension)
    problem['outliers'] = sorted(random.sample(range(problem['N']), int(np.array(num_outliers))))
    
    for i in problem['outliers']:
        y_gt = problem['y'][i]
        d = 0
        #print(OutlierSpace, np.shape(OutlierSpace))
        #print(d, chi2.ppf(0.999, int(MeasurementDimension)), d < chi2.ppf(0.999, int(MeasurementDimension)))
        while (d < chi2.ppf(0.999, int(MeasurementDimension))):
            #y_meas = rand_in(MeasurementDimension, 1, range(OutlierSpace[0], OutlierSpace[1]+1))
            
            y_meas = rand_in(MeasurementDimension, 1, (OutlierSpace))

            d = (y_gt - y_meas).T @ convi @ (y_gt - y_meas)
            #print(d < chi2.ppf(0.999, int(MeasurementDimension)))
        
        problem['y'][i] = y_meas
        
    problem['priors'] = []
    
    return problem
        
    
    

In [11]:
num_samples = 100; #% how many measurements are available
outliers_percentage = 0.6;

problem = linearRegressionProblem(num_samples, outliers_percentage);


In [12]:
import scipy.sparse as sp
from scipy.sparse.linalg import spsolve
from scipy import sparse



def leastSquareNorm2(problem, **kwargs):
    
    assert isProblem(problem) and problem['type']=='linear-estimation', 'You cannot solve this problem with linear least square'
    measurements = range(problem['N'])
    default_weights = np.ones(problem['N'])
    
    Inliers = kwargs.get('Inliers', measurements)
    assert all(isinstance(elem, numbers.Number) for elem in Inliers) and np.all(np.floor(Inliers)==Inliers), 'Inliers must be numeric and integers'
    #print('Inliers: ', Inliers)
    
    Weights = kwargs.get('Weights', default_weights)
    #print('Weights: ', Weights)
    #print('Type of Weights: ', type(Weights))
    
    assert all(isinstance(elem, numbers.Number) for elem in Weights), 'Weights must be numeric'
    #assert all(isinstance(elem, numbers.Number) for elem in Weights), "Weights must be numeric"
    
    Estimate = kwargs.get('Estimate', [])
    assert all(isinstance(elem, numbers.Number) for elem in Estimate), 'Estimate must be numeric'
    
    assert np.size(Weights)==problem['N'], 'The weights should be a 1xN vector'
    """
    A_ = {}
    for i in Inliers:
        A_[i] = (np.sqrt(Weights[i]) *  problem['A'][i])

    print(np.size(A_))
    print(A_)
    
    
    #A_ = np.vstack(A_)
    #A_sparse = sp.csr_matrix(A_)
    A_sparse = sparse.vstack(A_)
    #print(A_sparse)
    
    y_ = {}
    for i in Inliers:
        y_[i] = np.sqrt(Weights[i]*problem['y'][i])
    
    print(np.size(y_))
    print(y_)
    
    #y_ = np.vstack(y_)
    #y_sparse = sp.csr_matrix(y_)
    y_sparse = sparse.vstack(y_)
    #print(y_sparse)
    """
    
    A_ = [None] * len(Inliers)
    #print(np.size(A_))
    for i in Inliers:
        A_.append(np.sqrt(Weights[i]) * problem['A'][i])
    A_sparse = sparse.vstack(A_)

    y_ = [None] * len(Inliers)
    for i in Inliers:
        y_.append(np.sqrt(Weights[i]) * problem['y'][i])
    y_sparse = sparse.vstack(y_)
    
    #print(np.size(A_), np.size(y_))
    
    
    if np.size(Estimate)==0:
        #x = (A_sparse.T @ A_sparse)\(A_sparse.T @ y_sparse)
        #x = spsolve(A_.T @ A_, A_.T @ y_)
        x = spsolve(A_sparse.T @ A_sparse, A_sparse.T @ y_sparse)
    else:
        print('Not isempty(params.Results.Estimate)')
        x = kwargs['Estimate']
    
    #f_val = np.linalg.norm(np.dot(A_sparse.T , x) - y_sparse)**2
    #print(x, np.shape(x))
    
    f_val = np.linalg.norm(A_sparse.dot(np.array(x).reshape(-1, 1)) - y_sparse)**2
    
    residues = np.zeros(((int(np.array(problem['MeasurementDimension'])), problem['N'])))
    for i in range(problem['N']):
        #print(np.shape(problem['A'][i]))
        #print(np.shape(x))
        #print(np.shape(problem['y'][i]))
        #residues[:,i] = (problem['A'][i].dot(x.reshape(-1, 1)) - problem['y'][i]).ravel()
        residues[:,i] = (problem['A'][i].dot(x).reshape(-1,1) - problem['y'][i]).ravel()
        
    residuals = np.zeros(problem['N'])
    for i in range(problem['N']):
        residuals[i] = np.linalg.norm(residues[:,i])**2
        
    info = {'x': x, 'residuals': residuals, 'residues': residues}
    
    return f_val, info
    

In [13]:
problem = linearRegressionProblem(num_samples, outliers_percentage);
problem['f'] = leastSquareNorm2;
problem['MinMeasurements'] = 1;

In [14]:
def printProblemSummary(problem):
    num_outliers = np.size(problem['outliers']);
    print('Problem:\n');
    print('Type:', problem['type']);
    print('Measurements (N):', problem['N']);
    print('Num. outliers (k): ', num_outliers);
    print('Num. priors:   ', len(problem['priors']));
    print('Num. possible outliers (|V|): ', problem['N'] - np.size(problem['priors']));
    print('   k/N: %.03f\n', num_outliers/problem['N']);
    print('   k/|V|: %.03f\n', num_outliers/(problem['N'] - len(problem['priors'])));
    

In [15]:
printProblemSummary(problem)

Problem:

Type: linear-estimation
Measurements (N): 100
Num. outliers (k):  60
Num. priors:    0
Num. possible outliers (|V|):  100
   k/N: %.03f
 0.6
   k/|V|: %.03f
 0.6


In [16]:
print('  - Least-Square\n')
start_t = datetime.datetime.now();

[f_val_ls, info_ls] = leastSquareNorm2(problem, Inliers =range(problem['N']));
runtime = (datetime.datetime.now()-start_t)#*24*60*60;

  - Least-Square



In [17]:
print(runtime)

0:00:00.045959


In [18]:
print(f_val_ls)

663411.921101114


In [19]:
import numpy as np
from scipy.stats import chi2

cfg                 = {}
cfg['epsilon']      = chi2.ppf(0.99, problem['dof']) * np.array(problem['MeasurementNoiseStd'])**2
cfg['epsilon_mts']  = lambda n: chi2.ppf(0.99, n*problem['dof'])
cfg['epsilon_dmts'] = lambda n1, n2: UDGinv(0.99, n1*problem['dof']/2, n2*problem['dof']/2, 2*problem['MeasurementNoiseStd']**2, 2*problem['MeasurementNoiseStd']**2)
results             = {}
print('Solving with:')

Solving with:


In [20]:
print('Solving with:\n')

results = {}
results['ls'] = {}
results['ls']['algname'] = 'Least-Square'
results['ls']['f_val'] = f_val_ls
#tp, fp = detectionStats(problem, np.arange(1, problem['N']+1))
#results['ls']['stats'] = {'tp': tp, 'fp': fp}
results['ls']['iterations'] = 1
results['ls']['time'] = runtime


Solving with:



In [21]:
import scipy.sparse as sp
from scipy.sparse.linalg import spsolve
from scipy import sparse



def leastSquareNorm2(problem, **kwargs):
    
    assert isProblem(problem) and problem['type']=='linear-estimation', 'You cannot solve this problem with linear least square'
    measurements = range(problem['N'])
    default_weights = np.ones(problem['N'])
    
    Inliers = kwargs.get('Inliers', measurements)
    assert all(isinstance(elem, numbers.Number) for elem in Inliers) and np.all(np.floor(Inliers)==Inliers), 'Inliers must be numeric and integers'
    #print('Inliers: ', Inliers)
    
    Weights = kwargs.get('Weights', default_weights)
    #print('Weights: ', Weights)
    #print('Type of Weights: ', type(Weights))
    
    assert all(isinstance(elem, numbers.Number) for elem in Weights), 'Weights must be numeric'
    #assert all(isinstance(elem, numbers.Number) for elem in Weights), "Weights must be numeric"
    
    Estimate = kwargs.get('Estimate', [])
    assert all(isinstance(elem, numbers.Number) for elem in Estimate), 'Estimate must be numeric'
    
    assert np.size(Weights)==problem['N'], 'The weights should be a 1xN vector'
    """
    A_ = {}
    for i in Inliers:
        A_[i] = (np.sqrt(Weights[i]) *  problem['A'][i])

    print(np.size(A_))
    print(A_)
    
    
    #A_ = np.vstack(A_)
    #A_sparse = sp.csr_matrix(A_)
    A_sparse = sparse.vstack(A_)
    #print(A_sparse)
    
    y_ = {}
    for i in Inliers:
        y_[i] = np.sqrt(Weights[i]*problem['y'][i])
    
    print(np.size(y_))
    print(y_)
    
    #y_ = np.vstack(y_)
    #y_sparse = sp.csr_matrix(y_)
    y_sparse = sparse.vstack(y_)
    #print(y_sparse)
    """
    
    A_ = [None] * len(Inliers)
    #print(np.size(A_))
    for i in Inliers:
        A_.append(np.sqrt(Weights[i]) * problem['A'][i])
    A_sparse = sparse.vstack(A_)

    y_ = [None] * len(Inliers)
    for i in Inliers:
        y_.append(np.sqrt(Weights[i]) * problem['y'][i])
    y_sparse = sparse.vstack(y_)
    
    #print(np.size(A_), np.size(y_))
    
    """"
    if np.size(Estimate)==0:
        #x = (A_sparse.T @ A_sparse)\(A_sparse.T @ y_sparse)
        #x = spsolve(A_.T @ A_, A_.T @ y_)
        x = spsolve(A_sparse.T @ A_sparse, A_sparse.T @ y_sparse)
    else:
        print('Not isempty(params.Results.Estimate)')
        x = kwargs['Estimate']
    """
    
    if 'Estimate' in kwargs and kwargs['Estimate'] is not None:
        x = kwargs['Estimate']
    else:
        x = spsolve(A_sparse.T @ A_sparse, A_sparse.T @ y_sparse)
    
    
    #f_val = np.linalg.norm(np.dot(A_sparse.T , x) - y_sparse)**2
    #print(x, np.shape(x))
    
    f_val = np.linalg.norm(A_sparse.dot(np.array(x).reshape(-1, 1)) - y_sparse)**2
    
    residues = np.zeros(((int(np.array(problem['MeasurementDimension'])), problem['N'])))
    for i in range(problem['N']):
        #print(np.shape(problem['A'][i]))
        #print(np.shape(x))
        #print(np.shape(problem['y'][i]))
        #residues[:,i] = (problem['A'][i].dot(x.reshape(-1, 1)) - problem['y'][i]).ravel()
        residues[:,i] = (problem['A'][i].dot(x).reshape(-1,1) - problem['y'][i]).ravel()
        
    residuals = np.zeros(problem['N'])
    for i in range(problem['N']):
        residuals[i] = np.linalg.norm(residues[:,i])**2
        
    info = {'x': x, 'residuals': residuals, 'residues': residues}
    
    return f_val, info
    

In [22]:
print('  - Ground-Truth Least-Square...\n')
inliers_gt = np.setdiff1d(range(problem['N']), problem['outliers']);

  - Ground-Truth Least-Square...



In [23]:
start_t = datetime.datetime.now()
f_val_gt,info_gt = leastSquareNorm2(problem, Inliers=list(inliers_gt))
runtime = (datetime.datetime.now()-start_t)#*24*60*60;

In [24]:
print(inliers_gt)
print(np.where(info_gt['residuals']<10))

[ 0  8 10 13 21 23 25 27 28 29 30 33 39 42 43 44 47 48 49 50 51 58 59 60
 61 64 67 69 73 74 75 79 81 85 86 89 92 93 94 96]
(array([ 0,  8, 10, 13, 21, 23, 25, 27, 28, 29, 30, 33, 39, 42, 43, 44, 47,
       48, 49, 50, 51, 58, 59, 60, 61, 64, 67, 69, 73, 74, 75, 79, 81, 85,
       86, 89, 92, 93, 94, 96], dtype=int64),)


In [25]:
print(inliers_gt)
print(np.where(info_gt['residuals']<10))
print(info_gt['residuals']<8)

[ 0  8 10 13 21 23 25 27 28 29 30 33 39 42 43 44 47 48 49 50 51 58 59 60
 61 64 67 69 73 74 75 79 81 85 86 89 92 93 94 96]
(array([ 0,  8, 10, 13, 21, 23, 25, 27, 28, 29, 30, 33, 39, 42, 43, 44, 47,
       48, 49, 50, 51, 58, 59, 60, 61, 64, 67, 69, 73, 74, 75, 79, 81, 85,
       86, 89, 92, 93, 94, 96], dtype=int64),)
[ True False False False False False False False  True False  True False
 False  True False False False False False False False  True False  True
 False  True False  True  True  True  True False False  True False False
 False False False  True False False  True  True  True False False  True
  True  True  True  True False False False False False False  True  True
  True  True False False  True False False  True False  True False False
 False  True  True  True False False False  True False  True False False
 False  True  True False False  True False False  True  True  True False
  True False False False]


In [26]:
print(f_val_ls)
print(f_val_gt)

663411.921101114
2.6842471414342216e-27


In [27]:
def detectionStats(problem, inliers):
    num_outliers = len(problem['outliers'])
    if num_outliers == 0:
        true_positive_rate = 1
        false_positive_rate = 0
        precision = 1
        recall = 1
    else:
        outliers = np.setdiff1d(np.arange(1, problem['N']+1), inliers)
        true_inliers = np.setdiff1d(np.arange(1, problem['N']+1), np.union1d(problem['priors'], problem['outliers']))
        if len(true_inliers) == 0:
            true_positive_rate = 0
            false_positive_rate = 0
            precision = 0
            recall = 0
        else:
            true_positive = np.intersect1d(problem['outliers'], outliers)
            false_positive = np.intersect1d(true_inliers, outliers)
            true_positive_rate = len(true_positive) / num_outliers
            false_positive_rate = len(false_positive) / len(true_inliers)
            precision = len(true_positive) / (len(true_positive) + len(false_positive))
            recall = len(true_positive) / num_outliers
    return true_positive_rate, false_positive_rate, precision, recall


In [28]:
#tp, fp = detectionStats(problem, range(problem['N']))
#results['ls']['stats'] = {'tp': tp, 'fp': fp}

In [29]:
detectionStats(problem, range(problem['N']))

(0.0, 0.025, 0.0, 0.0)

In [30]:
results['gt'] = {}
results['gt']['algname'] = 'Ground-Truth';
results['gt']['f_val'] = f_val_gt;
#[results['gt'].stats.tp, results.gt.stats.fp] = detectionStats(problem, inliers_gt);
results['gt']['iterations'] = 1;
results['gt']['time'] = runtime;

In [31]:
print(results['gt']['f_val'])
print(results['ls']['f_val'])

2.6842471414342216e-27
663411.921101114


In [32]:
def gncWeightsUpdate(weights, mu, residuals, barc2):
    # GNC - GNC weights update for the TLS cost function
    # Reference:
    #    - "Graduated Non-Convexity for Robust Spatial Perception: From Non-Minimal Solvers to Global Outlier Rejection"
    #      Yang, Antonante, Tzoumas, Carlone (2020). IEEE Robotics and Automation Letters (RA-L), 5(2), 1127–1134.
    #      https://arxiv.org/pdf/1909.08605.pdf
    #    - "Outlier-Robust Estimation: Hardness, Minimally-Tuned Algorithms, and Applications"
    #      Antonante, Tzoumas, Yang, Carlone (2020).
    #      https://arxiv.org/pdf/2007.15109.pdf

    # Author: Pasquale Antonante
    # email: antonap@mit.edu
    # Date: 2021-01-06

    th1 = (mu+1)/mu * barc2
    th2 = (mu)/(mu+1) * barc2  # th1 > th2
    
    #print('th1 and th2 are: \n', th1, th2)
    
    #display(np.shape(np.array(residuals)))
    #display(np.size(np.array(residuals)))
    
    for k in range(np.size(residuals)):
    #for k in range(np.shape(residuals)[1]):
        #print('Residual[k]', residuals[k])
        #residuals_k = np.squeeze(residuals[k])
        #display(k, np.shape(residuals), np.shape(residuals[k]), np.shape(th1), np.shape(th2))
        #display(residuals[k])
        #display('Residuals[k] is: ', residuals[k])
        if (residuals[k] - th1) >= 0:
            weights[k] = 0
            #print('residuals(k) - th1 >= 0: \n', residuals[k], th1 )
        elif (residuals[k] - th2) <= 0:
            weights[k] = 1
            #print('residuals(k) - th2 <= 0: \n', residuals[k], th2 )
        else:
            weights[k] = math.sqrt( barc2*mu*(mu+1)/residuals[k] ) - mu
            
            #print('weights[k] = math.sqrt( barc2*mu*(mu+1)/residuals[k] ) - mu', residuals[k], weights[k])
            assert 0 <= weights[k] <= 1, f'weights {weights[k]} calculation wrong!'
    
    #display(type(weights))

    return weights, th1, th2


In [33]:
import math  
def gnc_vanilla(problem, f, **kwargs):
    continuation_factor = kwargs.get('ContinuationFactor', 1.4)
    initial_mu = kwargs.get('InitialMu', 'auto')
    noise_bound = kwargs.get('NoiseBound', 0)
    max_iterations = kwargs.get('MaxIterations', 1e3)
    fix_priors_weights = kwargs.get('FixPriorsWeights', True)
    cost_threshold = kwargs.get('CostThreshold', 0)
    debug = kwargs.get('Debug', False)
    init_ = kwargs.get('init_', 0)

    max_iterations = int(max_iterations)

    if debug:
        residuals_history = []
        weights_history = []

    if 'init_' not in kwargs:
        try:
            _, f_info = f(problem)
            #print('f(problem): ', f(problem))
        except Exception as e:
            print(f'Error message: {str(e)}')
            raise Exception("Could not run the global solver")
        assert 'residuals' in f_info, 'f should compute residuals'
    else:
        f_info = kwargs['init_']

    barc2 = noise_bound
    weights = np.ones(problem['N'])
    
    #print('barc2 is: {}', barc2)
    #print('Max of residuals: {}', np.max(f_info['residuals']))
    #print('Residuals are: {}', f_info['residuals'])
    #display(type(weights))
    
    
    if isinstance(initial_mu, str) and initial_mu.lower() == 'auto':
        mu = 1 / (2 * np.max(f_info['residuals']) / barc2 - 1 )
        #display('mu = 1 / (2 * np.max(f_info[residuals]) / barc2 - 1 )')
    elif isinstance(initial_mu, (int, float)):
        mu = initial_mu
    else:
        raise ValueError('Invalid value for InitialMu')

    prev_f_cost = 0

    i = 1
    info = {}
    info['stopping'] = 'MaxIterations'  # Worst case stopping
    info['status'] = 1  # worst case status: failure

    while i < max_iterations:
        if debug:
            residuals_history.append(f_info['residuals'].tolist())
            weights_history.append(weights.tolist())
            
        #print('mu is: {}',mu)
        #print('f_info[residuals]')
        #print(f_info['residuals'])
        
        weights, th1out, th2out = gncWeightsUpdate(weights, mu, f_info['residuals'], barc2)
        #print('Weights after gnc algorithm')
        #print(weights)
        
        #print(type(weights))
        #print(type(list(weights)))
        
        if fix_priors_weights:
            weights[problem['priors']] = 1
        
        try:
            _, f_info = f(problem, Weights=weights)
            #print('_, f_info = f(problem, Weights=weights)')
            #print(weights)
            
        except Exception as e:
            print(f'Error message: {str(e)}')
            raise Exception("Could not run the global solver")
        
        f_cost = np.sum(f_info['residuals'] * weights)
        
        cost_diff = np.abs(f_cost - prev_f_cost)
        
        prev_f_cost = f_cost

        if (cost_diff < cost_threshold) or areBinaryWeights(weights):
            if debug:
                residuals_history.append(f_info['residuals'].tolist())
                weights_history.append(weights.tolist())
            info['residuals'] = f_info['residuals'].tolist()
            info['stopping'] = 'CostThreshold'
            info['status'] = 1
            break
        mu *= continuation_factor
        i += 1
    
    
    #print(np.where(weights > 1-2.2204e-16))
    
    inliers = np.where(weights > 1-2.2204e-16)[0].tolist()
    
    info['Iterations'] = i
    info['params'] = kwargs
    info['Algorithm'] = 'GNC'
    if debug:
        info['mu'] = mu
        info['barc2History'] = np.repeat(barc2, i+1)
        info['ResidualsHistory'] = residuals_history
        info['WeightsHistory'] = weights_history
        info['CostDiff'] = cost_diff
        info['AreBinaryWeights'] = areBinaryWeights(weights)

    return inliers, info


In [34]:
def gnc(problem, f, **kwargs):
    """
    %GNC - Graduated Non-Convexity
    % Implementation of: 
    %    - "Graduated Non-Convexity for Robust Spatial Perception: From Non-Minimal Solvers to Global Outlier Rejection"
    %      Yang, Antonante, Tzoumas, Carlone (2020). IEEE Robotics and Automation Letters (RA-L), 5(2), 1127–1134.
    %      https://arxiv.org/pdf/1909.08605.pdf
    %    - "Outlier-Robust Estimation: Hardness, Minimally-Tuned Algorithms, and Applications" 
    %      Antonante, Tzoumas, Yang, Carlone (2020).
    %      https://arxiv.org/pdf/2007.15109.pdf
    %
    % Syntax:  [inliers, info] = gnc(problem, f, ...)
    %
    % Inputs:
    %    problem - the stracture representing a generic problem (see EmptyProblem)
    %    f - a function_handler of a non-minimal solver for problem
    %
    % Options:
    %    - ContinuationFactor: continuation factor for mu
    %    - InitialMu: initial mu (default: auto)
    %    - NoiseBound: inlier threshold
    %    - MaxIterations: maximum number of iterations
    %    - FixPriorsWeights: fix priors weights to 1 across all iterations
    %    - CostThreshold: if weighted sum of squared residuals is below this value, GNC stops
    %    - Debug: whether or not to enable the debug information
    %
    % Outputs:
    %    inliers - the indices of the inliers
    %    info - structure containing extended information about the xecution
    % 
    % Example:
    %   problem = linearRegressionProblem(100, 0.8);
    %   [inliers, info] = gnc(problem, @leastSquareNorm2, ...
    %       'NoiseBound', chi2inv(0.99, problem.dof)*problem.MeasurementNoiseStd^2);

    % Author: Pasquale Antonante
    % email: antonap@mit.edu
    % Date: 2021-01-06
    """
    
    #inliers = kwargs.get('Inliers', measurements)
    #assert all(isinstance(elem, numbers.Number) for elem in inliers) and all(np.floor(inliers)==inliers), "Inliers must be numeric and integers"
    #print(inliers)
    #weights = kwargs.get('Weights', default_weights)
    
    
    #params = inputParser;
    #params.KeepUnmatched = true;
    #params.addParameter('NoiseBound', 0, @(x) isscalar(x) && x>=0 && isfinite(x));
    #arams.parse(varargin{:});
    #start_t = now;
    #if params.Results.NoiseBound > 0
    #    [inliers, info] = gnc_vanilla(problem, f, varargin{:});
    #else
    #    error('You need to set the Noise Bound')
    #end
    #end_t = now;
    
    NoiseBound = kwargs.get('NoiseBound', 0)
    assert(np.size(NoiseBound)==1 and NoiseBound >= 0 and np.isfinite(NoiseBound))
    
    assert isProblem(problem), 'The problem doesn''t contain required fields.'

    start_t = datetime.datetime.now();
    
    if NoiseBound > 0:
        [inliers, info] = gnc_vanilla(problem, f, **kwargs);
    else:
        ValueError('You need to set the Noise Bound')
    
    #print('Output of gnc_vanilla')
    #print(inliers)
    end_t = datetime.datetime.now();
    
    info = {}
    info['time'] = (end_t - start_t) #* 24 * 60 * 60; #% serial date number to sec
    
    return inliers, info
    


In [35]:
def areBinaryWeights(weights):
    # Check the input vector is (approximately) a binary vector
    flag = True
    th = 1e-12
    for i in range(len(weights)-1, -1, -1):
        if weights[i] > th and abs(1 - weights[i]) > th:
            flag = False
            break
    return flag

In [36]:
print('  - GNC\n')
[inliers_gnc, info_gnc] = gnc(problem, problem['f'], NoiseBound=cfg['epsilon']);

results['gnc'] = {}
results['gnc']['algname'] = 'GNC';
[f_val_gnc, info1_gnc] = leastSquareNorm2(problem, Inliers=inliers_gnc)

results['gnc']['f_val'] = f_val_gnc

#[results.gnc.stats.tp, results.gnc.stats.fp] = ...
#    detectionStats(problem, inliers);



  - GNC



In [37]:
print(results['ls']['f_val'])
print(results['gt']['f_val'])
print(results['gnc']['f_val'])

663411.921101114
2.6842471414342216e-27
2.6842471414342216e-27


In [38]:
print(inliers_gnc)

[0, 8, 10, 13, 21, 23, 25, 27, 28, 29, 30, 33, 39, 42, 43, 44, 47, 48, 49, 50, 51, 58, 59, 60, 61, 64, 67, 69, 73, 74, 75, 79, 81, 85, 86, 89, 92, 93, 94, 96]


In [39]:
print(inliers_gt)

[ 0  8 10 13 21 23 25 27 28 29 30 33 39 42 43 44 47 48 49 50 51 58 59 60
 61 64 67 69 73 74 75 79 81 85 86 89 92 93 94 96]


In [40]:
print(inliers_gnc == inliers_gt)

[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True]


In [41]:
print(info_gt['residues'])

[[ 8.88178420e-16  6.11466008e+00  4.75820610e+01 -4.65295375e+01
   8.80131636e+01 -9.74955383e+01  7.62404214e+01 -4.13344943e+01
   2.66453526e-15  4.24085375e+01 -2.66453526e-15 -6.94097551e+01
   3.55558793e+01 -5.32907052e-15 -1.10355200e+01 -1.12192331e+01
  -9.44088338e+01  3.89215975e+01 -6.14128852e+01 -5.09091234e+01
   1.22742469e+01 -4.44089210e-16 -5.21314460e+01 -1.77635684e-15
   9.34636302e+01  4.44089210e-15 -3.63871919e+01 -3.55271368e-15
  -2.44249065e-15  5.32907052e-15  2.22044605e-16  3.50733386e+01
   5.07051998e+01 -1.33226763e-15 -6.25876114e+01 -7.64686126e+01
  -2.97708104e+01 -3.28111048e+00  8.02164025e+01 -4.88498131e-15
   7.85742513e+01 -2.20379204e+01  5.32907052e-15  2.66453526e-15
   2.66453526e-15 -5.78090111e+01  9.10478363e+01  6.21724894e-15
  -5.32907052e-15  0.00000000e+00  0.00000000e+00 -3.55271368e-15
   5.52254952e+01  6.29394507e+01  3.91122095e+01  6.14501103e+01
   9.23103705e+01  7.70216552e+01 -3.55271368e-15 -1.77635684e-15
  -5.32907

In [42]:
print(info_gt['residuals'])

[3.23432971e-29 5.20418459e+03 7.72150382e+03 3.89076000e+03
 1.66149146e+04 2.47897750e+04 1.14177130e+04 8.35814368e+03
 2.38630424e-29 1.22908806e+04 6.38977333e-29 2.08294303e+04
 4.48617651e+03 9.54521695e-29 7.50272485e+03 6.97624385e+03
 1.32830925e+04 1.05162864e+04 1.08812251e+04 8.02516725e+03
 4.51277420e+03 5.85729222e-29 1.76150836e+04 1.23851162e-28
 1.32830532e+04 2.36658272e-29 1.20646360e+04 5.62063395e-29
 5.72417194e-29 3.72736778e-29 7.43994441e-29 1.15790567e+04
 1.10948350e+04 1.75521551e-29 1.37326231e+04 1.41647089e+04
 3.16985786e+03 2.32295651e+03 1.27642653e+04 4.00346909e-29
 1.67063071e+04 1.06104697e+04 2.99767144e-29 4.22040584e-29
 4.59511477e-29 8.80584118e+03 1.76776572e+04 1.00382550e-28
 3.33293732e-29 4.10207671e-29 1.10834957e-28 6.62643160e-29
 1.23057176e+04 1.30334025e+04 1.25511749e+04 1.18352710e+04
 2.36837017e+04 2.07925407e+04 3.94430453e-29 8.20415341e-29
 3.51536141e-29 1.33711923e-28 1.10349776e+04 1.50294278e+04
 1.11229388e-28 1.170501

In [43]:
print(info_gt['residuals'])

[3.23432971e-29 5.20418459e+03 7.72150382e+03 3.89076000e+03
 1.66149146e+04 2.47897750e+04 1.14177130e+04 8.35814368e+03
 2.38630424e-29 1.22908806e+04 6.38977333e-29 2.08294303e+04
 4.48617651e+03 9.54521695e-29 7.50272485e+03 6.97624385e+03
 1.32830925e+04 1.05162864e+04 1.08812251e+04 8.02516725e+03
 4.51277420e+03 5.85729222e-29 1.76150836e+04 1.23851162e-28
 1.32830532e+04 2.36658272e-29 1.20646360e+04 5.62063395e-29
 5.72417194e-29 3.72736778e-29 7.43994441e-29 1.15790567e+04
 1.10948350e+04 1.75521551e-29 1.37326231e+04 1.41647089e+04
 3.16985786e+03 2.32295651e+03 1.27642653e+04 4.00346909e-29
 1.67063071e+04 1.06104697e+04 2.99767144e-29 4.22040584e-29
 4.59511477e-29 8.80584118e+03 1.76776572e+04 1.00382550e-28
 3.33293732e-29 4.10207671e-29 1.10834957e-28 6.62643160e-29
 1.23057176e+04 1.30334025e+04 1.25511749e+04 1.18352710e+04
 2.36837017e+04 2.07925407e+04 3.94430453e-29 8.20415341e-29
 3.51536141e-29 1.33711923e-28 1.10349776e+04 1.50294278e+04
 1.11229388e-28 1.170501

In [44]:
num_samples = 100; #% how many measurements are available
outliers_percentage = 0.6;

problem = linearRegressionProblem(num_samples, outliers_percentage);

problem = linearRegressionProblem(num_samples, outliers_percentage);
problem['f'] = leastSquareNorm2;
problem['MinMeasurements'] = 1;

printProblemSummary(problem)

print('  - Least-Square\n')
start_t = datetime.datetime.now();

[f_val_ls, info_ls] = leastSquareNorm2(problem, Inliers =range(problem['N']));
runtime = (datetime.datetime.now()-start_t)#*24*60*60;

print('Solving with:\n')

results = {}
results['ls'] = {}
results['ls']['algname'] = 'Least-Square'
results['ls']['f_val'] = f_val_ls
#tp, fp = detectionStats(problem, np.arange(1, problem['N']+1))
#results['ls']['stats'] = {'tp': tp, 'fp': fp}
results['ls']['iterations'] = 1
results['ls']['time'] = runtime

print('  - Ground-Truth Least-Square...\n')
inliers_gt = np.setdiff1d(range(problem['N']), problem['outliers']);

start_t = datetime.datetime.now()
f_val_gt,info_gt = leastSquareNorm2(problem, Inliers=list(inliers_gt))
runtime_gt = (datetime.datetime.now()-start_t)#*24*60*60;

Problem:

Type: linear-estimation
Measurements (N): 100
Num. outliers (k):  60
Num. priors:    0
Num. possible outliers (|V|):  100
   k/N: %.03f
 0.6
   k/|V|: %.03f
 0.6
  - Least-Square

Solving with:

  - Ground-Truth Least-Square...



In [45]:
datetime.datetime.now()

datetime.datetime(2023, 5, 5, 22, 16, 47, 421923)

In [46]:
print(f_val_ls)
print(f_val_gt)

results['gt'] = {}
results['gt']['algname'] = 'Ground-Truth';
results['gt']['f_val'] = f_val_gt;
#[results['gt'].stats.tp, results.gt.stats.fp] = detectionStats(problem, inliers_gt);
results['gt']['iterations'] = 1;
results['gt']['time'] = runtime_gt;

print(results['gt']['f_val'])
print(results['ls']['f_val'])

633808.0487532781
3.786988405271688e-27
3.786988405271688e-27
633808.0487532781


In [47]:
print('  - GNC\n')
#start_t = datetime.datetime.now()
[inliers_gnc, info_gnc] = gnc(problem, problem['f'], NoiseBound=cfg['epsilon']);
#runtime_gnc = (datetime.datetime.now()-start_t)*24*60*60;

results['gnc'] = {}
results['gnc']['algname'] = 'GNC';
[f_val_gnc, info1_gnc] = leastSquareNorm2(problem, Inliers=inliers_gnc)

results['gnc']['f_val'] = f_val_gnc
results['gnc']['time'] = info_gnc['time'];

#[results.gnc.stats.tp, results.gnc.stats.fp] = ...
#    detectionStats(problem, inliers);



  - GNC



In [48]:
print(results['ls']['f_val'])
print(results['gt']['f_val'])
print(results['gnc']['f_val'])

633808.0487532781
3.786988405271688e-27
3.786988405271688e-27


In [49]:
print(results['ls']['time'])
print(results['gt']['time'])
print(results['gnc']['time'])

0:00:00.023302
0:00:00.008061
0:00:00.326636


In [50]:
num_samples = 100; #% how many measurements are available
outliers_percentage = 0.7;

problem = linearRegressionProblem(num_samples, outliers_percentage);

problem = linearRegressionProblem(num_samples, outliers_percentage);
problem['f'] = leastSquareNorm2;
problem['MinMeasurements'] = 1;

printProblemSummary(problem)

print('  - Least-Square\n')
start_t = datetime.datetime.now();

[f_val_ls, info_ls] = leastSquareNorm2(problem, Inliers =range(problem['N']));
runtime = (datetime.datetime.now()-start_t)#*24*60*60;

print('Solving with:\n')

results = {}
results['ls'] = {}
results['ls']['algname'] = 'Least-Square'
results['ls']['f_val'] = f_val_ls
#tp, fp = detectionStats(problem, np.arange(1, problem['N']+1))
#results['ls']['stats'] = {'tp': tp, 'fp': fp}
results['ls']['iterations'] = 1
results['ls']['time'] = runtime

print('  - Ground-Truth Least-Square...\n')
inliers_gt = np.setdiff1d(range(problem['N']), problem['outliers']);

start_t = datetime.datetime.now()
f_val_gt,info_gt = leastSquareNorm2(problem, Inliers=list(inliers_gt))
runtime_gt = (datetime.datetime.now()-start_t)#*24*60*60;

print(f_val_ls)
print(f_val_gt)

results['gt'] = {}
results['gt']['algname'] = 'Ground-Truth';
results['gt']['f_val'] = f_val_gt;
#[results['gt'].stats.tp, results.gt.stats.fp] = detectionStats(problem, inliers_gt);
results['gt']['iterations'] = 1;
results['gt']['time'] = runtime_gt;

print(results['gt']['f_val'])
print(results['ls']['f_val'])

print('  - GNC\n')
[inliers_gnc, info_gnc] = gnc(problem, problem['f'], NoiseBound=cfg['epsilon']);

results['gnc'] = {}
results['gnc']['algname'] = 'GNC';
[f_val_gnc, info1_gnc] = leastSquareNorm2(problem, Inliers=inliers_gnc)

results['gnc']['f_val'] = f_val_gnc
results['gnc']['time'] = info_gnc['time'];

#[results.gnc.stats.tp, results.gnc.stats.fp] = ...
#    detectionStats(problem, inliers);



Problem:

Type: linear-estimation
Measurements (N): 100
Num. outliers (k):  70
Num. priors:    0
Num. possible outliers (|V|):  100
   k/N: %.03f
 0.7
   k/|V|: %.03f
 0.7
  - Least-Square

Solving with:

  - Ground-Truth Least-Square...

641338.4718590766
8.476803464665535e-27
8.476803464665535e-27
641338.4718590766
  - GNC



In [51]:
print(inliers_gnc)
print(inliers_gt)
print(inliers_gnc == inliers_gt)
print(results['ls']['f_val'])
print(results['gt']['f_val'])
print(results['gnc']['f_val'])

[1, 5, 7, 8, 9, 10, 14, 16, 19, 23, 25, 30, 35, 43, 44, 53, 64, 69, 70, 71, 73, 74, 79, 81, 85, 88, 90, 92, 93, 99]
[ 1  5  7  8  9 10 14 16 19 23 25 30 35 43 44 53 64 69 70 71 73 74 79 81
 85 88 90 92 93 99]
[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True]
641338.4718590766
8.476803464665535e-27
8.476803464665535e-27


In [52]:
print(results['ls']['time'])
print(results['gt']['time'])
print(results['gnc']['time'])

0:00:00.018996
0:00:00.007577
0:00:00.288663


In [53]:
num_samples = 100; #% how many measurements are available
outliers_percentage = 0.8;

problem = linearRegressionProblem(num_samples, outliers_percentage);

problem = linearRegressionProblem(num_samples, outliers_percentage);
problem['f'] = leastSquareNorm2;
problem['MinMeasurements'] = 1;

printProblemSummary(problem)

print('  - Least-Square\n')
start_t = datetime.datetime.now();

[f_val_ls, info_ls] = leastSquareNorm2(problem, Inliers =range(problem['N']));
runtime = (datetime.datetime.now()-start_t)#*24*60*60;

print('Solving with:\n')

results = {}
results['ls'] = {}
results['ls']['algname'] = 'Least-Square'
results['ls']['f_val'] = f_val_ls
#tp, fp = detectionStats(problem, np.arange(1, problem['N']+1))
#results['ls']['stats'] = {'tp': tp, 'fp': fp}
results['ls']['iterations'] = 1
results['ls']['time'] = runtime

print('  - Ground-Truth Least-Square...\n')
inliers_gt = np.setdiff1d(range(problem['N']), problem['outliers']);

start_t = datetime.datetime.now()
f_val_gt,info_gt = leastSquareNorm2(problem, Inliers=list(inliers_gt))
runtime_gt = (datetime.datetime.now()-start_t)#*24*60*60;

print(f_val_ls)
print(f_val_gt)

results['gt'] = {}
results['gt']['algname'] = 'Ground-Truth';
results['gt']['f_val'] = f_val_gt;
#[results['gt'].stats.tp, results.gt.stats.fp] = detectionStats(problem, inliers_gt);
results['gt']['iterations'] = 1;
results['gt']['time'] = runtime_gt;

print(results['gt']['f_val'])
print(results['ls']['f_val'])

print('  - GNC\n')
[inliers_gnc, info_gnc] = gnc(problem, problem['f'], NoiseBound=cfg['epsilon']);

results['gnc'] = {}
results['gnc']['algname'] = 'GNC';
[f_val_gnc, info1_gnc] = leastSquareNorm2(problem, Inliers=inliers_gnc)

results['gnc']['f_val'] = f_val_gnc
results['gnc']['time'] = info_gnc['time'];
#[results.gnc.stats.tp, results.gnc.stats.fp] = ...
#    detectionStats(problem, inliers);



Problem:

Type: linear-estimation
Measurements (N): 100
Num. outliers (k):  80
Num. priors:    0
Num. possible outliers (|V|):  100
   k/N: %.03f
 0.8
   k/|V|: %.03f
 0.8
  - Least-Square

Solving with:

  - Ground-Truth Least-Square...

759699.3763140291
6.100113468654355e-29
6.100113468654355e-29
759699.3763140291
  - GNC



In [54]:
print(inliers_gnc)
print(inliers_gt)
print(inliers_gnc == inliers_gt)
print(results['ls']['f_val'])
print(results['gt']['f_val'])
print(results['gnc']['f_val'])

[8, 9, 12, 13, 14, 22, 26, 30, 31, 34, 35, 39, 42, 45, 65, 73, 75, 83, 88, 89]
[ 8  9 12 13 14 22 26 30 31 34 35 39 42 45 65 73 75 83 88 89]
[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True]
759699.3763140291
6.100113468654355e-29
6.100113468654355e-29


In [55]:
print(results['ls']['time'])
print(results['gt']['time'])
print(results['gnc']['time'])

0:00:00.021912
0:00:00.009660
0:00:00.320849


In [56]:
num_samples = 100; #% how many measurements are available
outliers_percentage = 0.9;

problem = linearRegressionProblem(num_samples, outliers_percentage);

problem = linearRegressionProblem(num_samples, outliers_percentage);
problem['f'] = leastSquareNorm2;
problem['MinMeasurements'] = 1;

printProblemSummary(problem)

print('  - Least-Square\n')
start_t = datetime.datetime.now();

[f_val_ls, info_ls] = leastSquareNorm2(problem, Inliers =range(problem['N']));
runtime = (datetime.datetime.now()-start_t)#*24*60*60;

print('Solving with:\n')

results = {}
results['ls'] = {}
results['ls']['algname'] = 'Least-Square'
results['ls']['f_val'] = f_val_ls
#tp, fp = detectionStats(problem, np.arange(1, problem['N']+1))
#results['ls']['stats'] = {'tp': tp, 'fp': fp}
results['ls']['iterations'] = 1
results['ls']['time'] = runtime

print('  - Ground-Truth Least-Square...\n')
inliers_gt = np.setdiff1d(range(problem['N']), problem['outliers']);

start_t = datetime.datetime.now()
f_val_gt,info_gt = leastSquareNorm2(problem, Inliers=list(inliers_gt))
runtime_gt = (datetime.datetime.now()-start_t)#*24*60*60;

print(f_val_ls)
print(f_val_gt)

results['gt'] = {}
results['gt']['algname'] = 'Ground-Truth';
results['gt']['f_val'] = f_val_gt;
#[results['gt'].stats.tp, results.gt.stats.fp] = detectionStats(problem, inliers_gt);
results['gt']['iterations'] = 1;
results['gt']['time'] = runtime_gt;

print(results['gt']['f_val'])
print(results['ls']['f_val'])

print('  - GNC\n')
[inliers_gnc, info_gnc] = gnc(problem, problem['f'], NoiseBound=cfg['epsilon']);

results['gnc'] = {}
results['gnc']['algname'] = 'GNC';
[f_val_gnc, info1_gnc] = leastSquareNorm2(problem, Inliers=inliers_gnc)

results['gnc']['f_val'] = f_val_gnc
results['gnc']['time'] = info_gnc['time'];
#[results.gnc.stats.tp, results.gnc.stats.fp] = ...
#    detectionStats(problem, inliers);



Problem:

Type: linear-estimation
Measurements (N): 100
Num. outliers (k):  90
Num. priors:    0
Num. possible outliers (|V|):  100
   k/N: %.03f
 0.9
   k/|V|: %.03f
 0.9
  - Least-Square

Solving with:

  - Ground-Truth Least-Square...

945277.1221616952
6.001505855501729e-29
6.001505855501729e-29
945277.1221616952
  - GNC



In [57]:
print(inliers_gnc)
print(inliers_gt)
print(inliers_gnc == inliers_gt)
print(results['ls']['f_val'])
print(results['gt']['f_val'])
print(results['gnc']['f_val'])

[25, 26, 31, 44, 46, 52, 68, 69, 88, 96]
[25 26 31 44 46 52 68 69 88 96]
[ True  True  True  True  True  True  True  True  True  True]
945277.1221616952
6.001505855501729e-29
6.001505855501729e-29


In [58]:
print(results['ls']['time'])
print(results['gt']['time'])
print(results['gnc']['time'])

0:00:00.038056
0:00:00.004977
0:00:00.310310


In [59]:
num_samples = 100; #% how many measurements are available
outliers_percentage = 0.6;

problem = linearRegressionProblem(num_samples, outliers_percentage);

problem = linearRegressionProblem(num_samples, outliers_percentage);
problem['f'] = leastSquareNorm2;
problem['MinMeasurements'] = 1;

printProblemSummary(problem)

print('  - Least-Square\n')
start_t = datetime.datetime.now();

[f_val_ls, info_ls] = leastSquareNorm2(problem, Inliers =range(problem['N']));
runtime = (datetime.datetime.now()-start_t)#*24*60*60;

print('Solving with:\n')

results = {}
results['ls'] = {}
results['ls']['algname'] = 'Least-Square'
results['ls']['f_val'] = f_val_ls
#tp, fp = detectionStats(problem, np.arange(1, problem['N']+1))
#results['ls']['stats'] = {'tp': tp, 'fp': fp}
results['ls']['iterations'] = 1
results['ls']['time'] = runtime

print('  - Ground-Truth Least-Square...\n')
inliers_gt = np.setdiff1d(range(problem['N']), problem['outliers']);

start_t = datetime.datetime.now()
f_val_gt,info_gt = leastSquareNorm2(problem, Inliers=list(inliers_gt))
runtime_gt = (datetime.datetime.now()-start_t)#*24*60*60;

print(f_val_ls)
print(f_val_gt)

results['gt'] = {}
results['gt']['algname'] = 'Ground-Truth';
results['gt']['f_val'] = f_val_gt;
#[results['gt'].stats.tp, results.gt.stats.fp] = detectionStats(problem, inliers_gt);
results['gt']['iterations'] = 1;
results['gt']['time'] = runtime;

print(results['gt']['f_val'])
print(results['ls']['f_val'])

print('  - GNC\n')
[inliers_gnc, info_gnc] = gnc(problem, problem['f'], NoiseBound=cfg['epsilon']);

results['gnc'] = {}
results['gnc']['algname'] = 'GNC';
[f_val_gnc, info1_gnc] = leastSquareNorm2(problem, Inliers=inliers_gnc)

results['gnc']['f_val'] = f_val_gnc
results['gnc']['time'] = info_gnc['time'];
#[results.gnc.stats.tp, results.gnc.stats.fp] = ...
#    detectionStats(problem, inliers);



Problem:

Type: linear-estimation
Measurements (N): 100
Num. outliers (k):  60
Num. priors:    0
Num. possible outliers (|V|):  100
   k/N: %.03f
 0.6
   k/|V|: %.03f
 0.6
  - Least-Square

Solving with:

  - Ground-Truth Least-Square...

596806.947325572
1.0348733414900065e-26
1.0348733414900065e-26
596806.947325572
  - GNC



In [60]:
print(inliers_gnc)
print(inliers_gt)
print(inliers_gnc == inliers_gt)
print(results['ls']['f_val'])
print(results['gt']['f_val'])
print(results['gnc']['f_val'])

[0, 5, 6, 9, 11, 12, 15, 18, 19, 20, 21, 22, 23, 26, 29, 32, 36, 37, 39, 41, 46, 49, 50, 53, 54, 59, 62, 65, 66, 68, 71, 73, 75, 76, 81, 85, 87, 90, 93, 98]
[ 0  5  6  9 11 12 15 18 19 20 21 22 23 26 29 32 36 37 39 41 46 49 50 53
 54 59 62 65 66 68 71 73 75 76 81 85 87 90 93 98]
[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True]
596806.947325572
1.0348733414900065e-26
1.0348733414900065e-26


In [61]:
print(results['ls']['time'])
print(results['gt']['time'])
print(results['gnc']['time'])

0:00:00.020379
0:00:00.020379
0:00:00.296332


In [62]:
num_samples = 100; #% how many measurements are available
outliers_percentage = 0.5;

problem = linearRegressionProblem(num_samples, outliers_percentage);

problem = linearRegressionProblem(num_samples, outliers_percentage);
problem['f'] = leastSquareNorm2;
problem['MinMeasurements'] = 1;

printProblemSummary(problem)

print('  - Least-Square\n')
start_t = datetime.datetime.now();

[f_val_ls, info_ls] = leastSquareNorm2(problem, Inliers =range(problem['N']));
runtime = (datetime.datetime.now()-start_t)#*24*60*60;

print('Solving with:\n')

results = {}
results['ls'] = {}
results['ls']['algname'] = 'Least-Square'
results['ls']['f_val'] = f_val_ls
#tp, fp = detectionStats(problem, np.arange(1, problem['N']+1))
#results['ls']['stats'] = {'tp': tp, 'fp': fp}
results['ls']['iterations'] = 1
results['ls']['time'] = runtime

print('  - Ground-Truth Least-Square...\n')
inliers_gt = np.setdiff1d(range(problem['N']), problem['outliers']);

start_t = datetime.datetime.now()
f_val_gt,info_gt = leastSquareNorm2(problem, Inliers=list(inliers_gt))
runtime_gt = (datetime.datetime.now()-start_t)#*24*60*60;

print(f_val_ls)
print(f_val_gt)

results['gt'] = {}
results['gt']['algname'] = 'Ground-Truth';
results['gt']['f_val'] = f_val_gt;
#[results['gt'].stats.tp, results.gt.stats.fp] = detectionStats(problem, inliers_gt);
results['gt']['iterations'] = 1;
results['gt']['time'] = runtime_gt;

print(results['gt']['f_val'])
print(results['ls']['f_val'])

print('  - GNC\n')
[inliers_gnc, info_gnc] = gnc(problem, problem['f'], NoiseBound=cfg['epsilon']);

results['gnc'] = {}
results['gnc']['algname'] = 'GNC';
[f_val_gnc, info1_gnc] = leastSquareNorm2(problem, Inliers=inliers_gnc)

results['gnc']['f_val'] = f_val_gnc
results['gnc']['time'] = info_gnc['time'];
#[results.gnc.stats.tp, results.gnc.stats.fp] = ...
#    detectionStats(problem, inliers);



Problem:

Type: linear-estimation
Measurements (N): 100
Num. outliers (k):  50
Num. priors:    0
Num. possible outliers (|V|):  100
   k/N: %.03f
 0.5
   k/|V|: %.03f
 0.5
  - Least-Square

Solving with:

  - Ground-Truth Least-Square...

476280.8836031935
1.5908699800979597e-26
1.5908699800979597e-26
476280.8836031935
  - GNC



In [63]:
print(inliers_gnc)
print(inliers_gt)
print(inliers_gnc == inliers_gt)
print(results['ls']['f_val'])
print(results['gt']['f_val'])
print(results['gnc']['f_val'])

[0, 2, 6, 8, 9, 11, 12, 14, 17, 21, 23, 25, 27, 28, 32, 33, 34, 35, 37, 40, 41, 43, 47, 48, 51, 53, 54, 57, 58, 59, 60, 63, 64, 65, 66, 67, 68, 72, 74, 76, 79, 82, 84, 85, 86, 87, 92, 93, 95, 97]
[ 0  2  6  8  9 11 12 14 17 21 23 25 27 28 32 33 34 35 37 40 41 43 47 48
 51 53 54 57 58 59 60 63 64 65 66 67 68 72 74 76 79 82 84 85 86 87 92 93
 95 97]
[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True]
476280.8836031935
1.5908699800979597e-26
1.5908699800979597e-26


In [64]:
print(results['ls']['time'])
print(results['gt']['time'])
print(results['gnc']['time'])

0:00:00.032272
0:00:00.021291
0:00:00.595251
